<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Brightway (2.5) Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Romain Sacchi</a> (PSI)
                    <br><br>
                    Duration: 1 hour 30 minutes.
                </center>
            </td>
        </tr>
    </div>

# Brightway I/O: bw2io

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2.5</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2</a>. From the user end side, very little differ between the two. The code executed throughout this notebook works with both versions.
</div>


## Learning objectives  

Learn how to:

    - input LCI data to Brightway using Excel/CVS importers
    - fix linking issue using migration files
    - do a contribution analysis
    - export your foreground inventories

## Standard inputs and setup

In [ ]:
import os
from pathlib import Path
import pandas as pd
import bw2io, bw2data, bw2calc

Let's list our projects:

In [ ]:
list(bw2data.projects)

Setting the project

In [ ]:
bw2data.projects.set_current("brightcon-2025")

## Context

Performing an LCA generally requires:
  - Background LCI data (e.g., an LCI database such as [ecoinvent](https://ecoinvent.org/))  
  - Foreground LCI data (e.g., a bunch of datasets the LCA practitioner has spent time modeling)
  - Sets of characterization factors.
 
This section will deal with the way Foreground LCI data is input to Brightway.

Useful documentation about what a database in Brightway is can be found [here](https://github.com/brightway-lca/brightway2/blob/master/notebooks/Databases.ipynb)
 and [here](https://docs.brightway.dev/en/latest/content/gettingstarted/databases.html).

# Importing from CSV or Excel

Using `bw2io.ExcelImporter`, we import datasets from an Excel file.

In [ ]:
imp = bw2io.ExcelImporter(Path(".") / "files" / "lci-carbon-fiber.xlsx")

We want to apply a number of data cleaning functions (format numbers, set correct location, etc.),

In [ ]:
imp.apply_strategies()

Then, we want to use the `match_database()` method to link exchanges to suppliers.

First, we want to link exchange to suppliers that may also be contained in the data being imported.

In [ ]:
# we match based on the name, reference product and location
imp.match_database(fields=('name', 'reference product', 'unit', 'location')) 

<div class="alert alert-info">
Note: Why is it important to link both based on <b>name</b> and <b>reference product</b>?
</div>

Is that enough? Do we still have unlinked exchanges? Let's check.

In [ ]:
imp.statistics()

Let's check what those unlinked exchanges are:

In [ ]:
for u in list(imp.unlinked):
    print(u["name"], u.get("location"), u.get("categories"))

In [ ]:
bw2data.databases

OK, some unlinked exchanges are clearly from ecoinvent. Let's try to link those.

In [ ]:
imp.match_database("ecoinvent-3.11-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.statistics()

Depiste trying to link with ecoinvent, we still have two unmatched technosphere flows:

In [ ]:
[u for u in imp.unlinked if u["type"] == "technosphere"]

Also, we have an unlinked biosphere exchange left, let's try to match that one.

In [ ]:
imp.match_database("biosphere", fields=('name', 'unit', 'categories'))
imp.statistics()

In [ ]:
[u for u in imp.unlinked if u["type"] == "biosphere"]

Nope. Why not? Maybe because `Argon-40` does not not exist as such in `biosphere?`

In [ ]:
[f for f in bw2data.Database("biosphere") if "argon" in f["name"].lower()]

It is indeed now simply called `Argon` in ecoinvent 3.10.
We can:
1. manually fix this (i.e., modify the exchange name in the Excel file),
2. go over `imp.data`(list), iterate through the exchanges and find `Argon-40` and replace it with `Argon`
3. create a `migration` file for translating ecoinvent 3.9 flows to 3.10

### Migration file

We create a mapping dictionary, and use it to create a `Migration` object.

In [ ]:
migration = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for ethylene glycol", "ethylene glycol", "GLO", ""),
            {"location": "RER",},
        ),
        (
            ("air separation, cryogenic", "nitrogen, liquid", "GLO", ""),
            {
                "name": "industrial gases production, cryogenic air separation",
                "location": "RER",
            },
        ),
        (
            ("air separation, cryogenic", "nitrogen, liquid", "RER", ""),
            {
                "name": "industrial gases production, cryogenic air separation",
                "location": "RER",
            },
        ),
        (
            ("Argon-40", "", "", ("air",)),
            {
                "name": "Argon",
            },
        )
    ],
}

In [ ]:
bw2io.Migration(name="ei3.9-3.11").write(data=migration, description="ei 3.9 to 3.11")

In [ ]:
"ei3.9-3.11" in bw2io.migrations

In [ ]:
bw2io.Migration("ei3.9-3.11")

We apply the migration on our imported data.

In [ ]:
imp.data = bw2io.strategies.migrate_exchanges(
    db=imp.data,
    migration="ei3.9-3.11"
)

In [ ]:
imp.match_database("ecoinvent-3.11-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.match_database("biosphere", fields=('name', 'unit', 'categories'))
imp.statistics()

We have zero unlinked exchanges, we're ready to write the database.

In [ ]:
if len(list(imp.unlinked)) == 0:
    imp.write_database()

# Contribution analyses

## Process contribution

We have already seen how to obtain a contribution analysis in terms of contributing processes:

In [ ]:
db = bw2data.Database("carbon fiber")

In [ ]:
# let's list the datasets in our new database "carbon fiber"
[a["name"] for a in db]

In [ ]:
activity = db.search('carbon fiber production, weaved, at factory')[0]
activity

In [ ]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [ ]:
lca = bw2calc.LCA({activity:1}, method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
rev_prod, rev_act, rev_bio = lca.reverse_dict()

In [ ]:
results_by_activity = (lca.characterized_inventory.sum(axis=0)).A1

In [ ]:
# Create a list of names in columns
list_of_names_in_columns = [
    bw2data.get_activity(rev_prod[col])['name'] 
    for col in range((lca.characterized_inventory.sum(axis=0)).shape[1])
]

In [ ]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

With BW2.5 But there is a simpler and more "official" way to obtain this.

In [ ]:
import bw2analyzer as ba

In [ ]:
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_processes(lca=lca)],
    columns=["score", "quantity", "name"]
)

## Importing custom impact assessment methods

Let's import an Excel file containing names of greenhouse gases and factors (according to the IPCC 2021 GWP method), and add a CF for Hydrogen (11 kg $CO_2$-eq./kg $H_2$)

In [ ]:
from bw2io import ExcelLCIAImporter
new_method = ExcelLCIAImporter(
    filepath="files/lcia_gwp2021_100a_w_bio.xlsx",
    name=("IPCC 2021", "GWP 100 with bio C and H2"),
    unit="kg CO2e",
    description="modified IPCC GWP100 method"
)

In [ ]:
new_method.apply_strategies()

In [ ]:
new_method.name = ("IPCC 2021", "GWP 100 with bio C and H2")

In [ ]:
new_method.write_methods(
    overwrite=True,
    verbose=True,
)

In [ ]:
("IPCC 2021", "GWP 100 with bio C and H2") in bw2data.methods

In [ ]:
act=bw2data.Database("ecoinvent-3.10-cutoff").random()
for method in [
    ('IPCC 2021', 'climate change', 'global warming potential (GWP100)'),
    ("IPCC 2021", "GWP 100 with bio C and H2")
]:
    test = bw2calc.LCA({act: 1}, method)
    test.lci()
    test.lcia()
    print(test.score)

We can also build a method manually.

In [ ]:
data = [
    [f.key, 1]
    for f in bw2data.Database("biosphere")
    if "carbon dioxide" in f["name"].lower()
]

In [ ]:
my_method = bw2data.Method(("IPCC 2021", "Simple CO2 IA method"))
my_method.validate(data)
my_method.metadata = {"unit": "kg CO2eq.", "abbreviation": "GWP"}
my_method.register()
my_method.write(data)


In [ ]:
method = ("IPCC 2021", "Simple CO2 IA method")
method in bw2data.methods

In [ ]:
test = bw2calc.LCA({act: 1}, method)
test.lci()
test.lcia()
print(test.score)

## Tree map

In [ ]:
from polyviz import treemap

In [ ]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
treemap(
    activity=activity,
    method=method
)

## Supply chain traversal

In [ ]:
from polyviz import sankey

Github repo: [link](https://github.com/romainsacchi/polyviz). No proper documentation yet, but a notebook with [examples](https://github.com/romainsacchi/polyviz/blob/main/examples/examples.ipynb).

In [ ]:
_, df = sankey(
    activity=activity,
    level=4,
    cutoff=0.01,
    method=method,
    labels_swap={
        "carbon fiber": "cf.",
        "production": "prod."
    }
)

## Violin plot

In [ ]:
from polyviz import violin
import warnings
warnings.filterwarnings("ignore")

In [ ]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
violin(
    activities=[
        a for a in bw2data.Database("ecoinvent-3.10-cutoff") 
        if a["unit"] == "ton kilometer"
    ][:3],
    method=method,
    iterations=100
)

# Exporting databases

## To Excel

We can export the entire database inventory to an Excel file.

In [ ]:
bw2io.export.write_lci_excel(
    database_name="carbon fiber",
    dirpath=".",
)

## As a bw2package file

We can also export the database as a Brightway package file.

In [ ]:
bw2io.package.BW2Package().export_obj(
    obj=db,
)

<div class="alert alert-info">
Note: It may not be ideal for sharing, because for the import to be successful, the other user will need the databases the exported database depends on (ecoinvent, biosphere) to be named exactly the same. It is still possible, though, but the user you share the package with, will have to correct this upon import.
</div>

## As a project

We can export the entire project. This is the safest option, as all the database `carbon fiber` depends on are also exported. The drawback is that the file is bigger, and there may be licensing issues. But it is at least a good backup solution.

In [ ]:
bw2data.projects.current

In [ ]:
bw2io.backup.backup_project_directory(
    project='bw2',
    dir_backup='.' # here
)

And we load it back up... Note that I give it another name to not overwrite it.
Also, overwrite is `False` by default, so it would need ot be set to `True` first.

In [ ]:
bw2io.backup.restore_project_directory(
    fp="brightway2-project-bw2-backup.tar.gz",
    project_name="carbon fiber 2",
    overwrite_existing=False
)

Let's check

In [ ]:
"carbon fiber 2" in bw2data.projects